<a href="https://colab.research.google.com/github/marceloasr94/Coursera_Capstone/blob/main/Coursera_Project_Week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Introduction:

The city of Rio de Janeiro/Brazil is very big and has more than 150 "neighborhood", if one is moving from an specific neighborhood in São Paulo to Rio de Janeiro and wants to live in a neighborhood that at least looks like the one she/he lived before, where should they search places?

##Problem:

One is moving from the neighborhood "Moema", in São Paulo and never been in Rio de Janeiro before, this person works and study "Home Office" so he/she doesn't care for work/university distance. Which neighborhoods he/she should search in Rio de Janeiro? Which are the most looks like neighborhoods?
So We will separate neighborhood by Cluster and compare which cluster has more "Common Venues" to Moema, São Paulo. With this one will have a leaner list of neighborhoods to look.

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import numpy as np

In [2]:
!pip install geocoder

     |████████████████████████████████| 102kB 5.1MB/s 


In [3]:
import geocoder
from geopy.geocoders import Nominatim

Today the person Lives in Moema, a Neighborhood in São Paulo:

In [4]:
address = 'Moema, Sao Paulo, SP'

label="Moema"
geolocator = Nominatim(user_agent="project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Moema/SP are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Moema/SP are -23.597085, -46.6628884.


In [5]:
mapa_sp=folium.Map(location=[latitude, longitude], zoom_start=15)
moema=folium.map.FeatureGroup()
moema.add_child(folium.CircleMarker(
            [latitude, longitude],
            radius=200,
            popup=label,
            color='blue',
            fill_color='#3186cc',
            fill_opacity=0.4))
mapa_sp.add_child(moema)      
mapa_sp

In [6]:
col=['Neighbourhood', 'Latitude', 'Longitude']
data=[[label, latitude, longitude]]
sp_df = pd.DataFrame(data, columns=col)
sp_df


,Neighbourhood,Latitude,Longitude
0,Moema,-23.597085,-46.662888


The city we will be studying is Rio de Janeiro, let's get the Data from a Wikipedia page. The Page classifies each neighborhood by HDI and has a lot of columns that does not matter to us, let's get rid of them:

In [7]:
pd.set_option('display.max_rows', 140)
rj = pd.read_html("https://pt.wikipedia.org/wiki/Lista_de_bairros_do_Rio_de_Janeiro_por_IDH",header=1)[0]
rj = rj.drop(rj.columns[[0,2,3,4,5,6,7,8,9]], axis=1)
rj=rj.rename(columns={"Bairro ou grupo de bairros":"Neighbourhood"})
rj

,Neighbourhood
0,"Barra da Tijuca, Joá"
1,Gávea
2,Leblon
3,Jardim Guanabara
4,Ipanema
5,Lagoa
6,Flamengo
7,Humaitá
8,Laranjeiras
9,Jardim Botânico


In [8]:
rj.dtypes

Neighbourhood    object
dtype: object

As our data originated from a Wikipedia page that classifies each neighbourhood by its human development index(HDI), some of the neighbourhoods has the same position, so they are at the same ROW, separeted by commas, let's explode it...
Ex: In Row 7 I Have "Barra da Tijuca" and "Joá"

In [9]:
#Função para Separar Linhas com ","
def splitDataFrameList(df,target_column,separator):
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [10]:
new_rj=splitDataFrameList(rj,"Neighbourhood",",")

In [11]:
pd.set_option('display.max_rows', 165)
new_rj=new_rj.drop([76,99,110,122])
new_rj=new_rj.reset_index()

In [12]:
new_rj=new_rj.drop("index", axis=1)
new_rj

,Neighbourhood
0,Barra da Tijuca
1,Joá
2,Gávea
3,Leblon
4,Jardim Guanabara
5,Ipanema
6,Lagoa
7,Flamengo
8,Humaitá
9,Laranjeiras


Now I may Capture each Neighbourhood Longitude and Latitude using Geolocator:

In [13]:
new_rj['Latitude']=0.1
new_rj['Longitude']=0.1
for i,label in enumerate(new_rj['Neighbourhood']):
  geolocator = Nominatim(user_agent="project")
  location = geolocator.geocode(label+", Rio de Janeiro,RJ")
  latitude = location.latitude
  longitude = location.longitude
  new_rj['Latitude'][i]=latitude
  new_rj['Longitude'][i]=longitude
  print('The',i,' Lat and Long are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


The 0  Lat and Long are -22.9997404, -43.3659929.
The 1  Lat and Long are -23.0077797, -43.2872344.
The 2  Lat and Long are -22.9814243, -43.2383245.
The 3  Lat and Long are -22.983556, -43.2249377.
The 4  Lat and Long are -22.8128362, -43.2007792.
The 5  Lat and Long are -22.9839557, -43.2022163.
The 6  Lat and Long are -22.9624658, -43.2024884.
The 7  Lat and Long are -22.9339841, -43.1745744.
The 8  Lat and Long are -22.9546413, -43.2004797.
The 9  Lat and Long are -22.9343173, -43.1878165.
The 10  Lat and Long are -22.968384550000003, -43.22869449858106.
The 11  Lat and Long are -22.9719642, -43.1843432.
The 12  Lat and Long are -22.961704, -43.1669042.
The 13  Lat and Long are -22.9515096, -43.1862969.
The 14  Lat and Long are -22.954074, -43.1679727.
The 15  Lat and Long are -22.912091949999997, -43.23114540590559.
The 16  Lat and Long are -22.902953, -43.5591295.
The 17  Lat and Long are -22.4523313, -43.1350743.
The 18  Lat and Long are -22.9026425, -43.2817263.
The 19  Lat and

Finally my Study Data will be:

In [14]:
new_rj

,Neighbourhood,Latitude,Longitude
0,Barra da Tijuca,-22.999740,-43.365993
1,Joá,-23.007780,-43.287234
2,Gávea,-22.981424,-43.238324
3,Leblon,-22.983556,-43.224938
4,Jardim Guanabara,-22.812836,-43.200779
5,Ipanema,-22.983956,-43.202216
6,Lagoa,-22.962466,-43.202488
7,Flamengo,-22.933984,-43.174574
8,Humaitá,-22.954641,-43.200480
9,Laranjeiras,-22.934317,-43.187816


In [15]:
address_rj = 'Rio de Janeiro, RJ'

label="RJ"
geolocator = Nominatim(user_agent="project")
location_rj = geolocator.geocode(address_rj)
latitude_rj = location_rj.latitude
longitude_rj = location_rj.longitude
print('The geograpical coordinate of Rio De Janeiro/RJ are {}, {}.'.format(latitude_rj, longitude_rj))


The geograpical coordinate of Rio De Janeiro/RJ are -22.9110137, -43.2093727.


I May Plot each Neighbourhood in the map:

In [16]:
map_rj = folium.Map(location=[latitude_rj, longitude_rj], zoom_start=11)
# add markers to map
for lat, lng, label in zip(new_rj['Latitude'].values, new_rj['Longitude'].values, new_rj['Neighbourhood'].values):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rj)  
    
map_rj

So for this Week, that's all... For next Week We will discover which are the neighborhoods(Or cluster of neighborhoods) that our Guy/Girl should look places to live.